# Funções, bibliotecas e Dataframe ficticios

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.font_manager as fm #para alterar as fontes dos graficos
import seaborn as sns

import pyperclip

# Definindo o estilo de fonte e tema
fonte = fm.FontProperties(family="Calibri", style="italic")

sns.set_style('whitegrid') #cor da grade(fundo)

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
import traceback

In [4]:
import requests

# CARREGANDO BASE
arquivo = 'fake_database'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
print(url)
#response = requests.get(url); code = response.text; exec(code)
#df = fake_database2(2250); df_bck = df.copy(); display(df.head())

arquivo = 'funcoes_estatisticas'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
print(url)
response = requests.get(url); code = response.text; exec(code)

# Função para avaliação de modelos exibindo metricas de avaliação
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay
arquivo = 'ML_supervised_learning'
url = "https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/" + arquivo + ".py"
print(url); response = requests.get(url); code = response.text; exec(code)

print('TUDO OK')

https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/fake_database.py
https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/funcoes_estatisticas.py
https://raw.githubusercontent.com/GabrielGabes/functions_gsa/main/ML_supervised_learning.py
TUDO OK


In [5]:
def contribuicao_proba_tabela(df, coluna_analisada, model, sep=10):
    # Criar DataFrame para análise com a variável de interesse variando em um intervalo
    minimo, maximo = df[coluna_analisada].min(), df[coluna_analisada].max()
    df_analise_feature = pd.DataFrame()

    # Preencher as demais colunas com o valor médio
    for coluna in df.columns:
        if coluna != coluna_analisada:
            df_analise_feature[coluna] = df[coluna].mean()
        else:
            df_analise_feature[coluna] = np.linspace(minimo, maximo, sep)

    # Fazer previsão de probabilidade e adicionar ao DataFrame de análise
    df_analise_feature['predict_proba'] = model.predict_proba(df_analise_feature)[:, 1]

    # Retornar apenas a coluna analisada e a probabilidade prevista
    return df_analise_feature[[coluna_analisada, 'predict_proba']]

#x_data = pd.DataFrame(x_teste, columns=df.columns[:-1])
#resultado = contribuicao_proba_tabela(x_data, 'feature_4', modelo_m1, sep=10)
#resultado

def contribuicao_proba_grafico(resultado, coluna):
    cmap = plt.get_cmap('viridis')
    norm = plt.Normalize(resultado['predict_proba'].min(), resultado['predict_proba'].max())
    colors = cmap(norm(resultado['predict_proba']))

    ax = sns.lineplot(x=resultado[coluna].round(1), y=resultado['predict_proba'])

#plt.figure(figsize=(4,3))
#contribuicao_proba_grafico(resultado, 'feature_4')
#plt.show()

def contribuicao_proba(x_data, modelo, sep=10):
    for i, coluna_analisada in enumerate(x_data.columns, 1):
        plt.subplot(10, 3, i)
        resultado = contribuicao_proba_tabela(x_data, coluna_analisada, modelo, sep=sep)
        contribuicao_proba_grafico(resultado, coluna_analisada)


from sklearn.inspection import PartialDependenceDisplay
def contribuicao_proba2(x_data, modelo):
    # Definir o tamanho da figura geral
    fig, axes = plt.subplots(len(x_data.columns)//3+1, 3, figsize=(10, 10))
    axes = axes.ravel()  # Transformar a matriz de eixos em uma lista para fácil iteração

    # Iterar pelas colunas e gerar gráficos de dependência parcial
    for i, column in enumerate(x_data.columns):
        PartialDependenceDisplay.from_estimator(modelo, x_data, [column], ax=axes[i])

    # Ajustar o layout para evitar sobreposição
    plt.tight_layout()
    plt.show()

# Criando dataframe sintetico desbalanceado

In [6]:
from sklearn.datasets import make_classification

# Gerar dados sintéticos desbalanceados
x, y = make_classification(n_samples=7346, 
                           n_features=10, # qtd total de features
                           n_informative=6, # features realmente uteis para o modelo
                           n_redundant=3, # features que diz praticamente oque uma outra já fim
                           n_classes=2, 
                           n_clusters_per_class=3,  # Subgrupos
                           weights=[0.95, 0.05], 
                           class_sep=0.125,
                           random_state=3141592)

# Criar um DataFrame
df = pd.DataFrame(x, columns=[f'feature_{i}' for i in range(x.shape[1])])
df['target'] = y

# Ver a distribuição de classes
print(df['target'].value_counts(normalize=True))
print(df['target'].value_counts())

display(pd.concat([df.head(5), df.tail(5)]).T)
# Salvar para análise
#df.to_csv('imbalanced_dataset.csv', index=False)


target
0    0.945685
1    0.054315
Name: proportion, dtype: float64
target
0    6947
1     399
Name: count, dtype: int64


,0,1,2,3,4,7341,7342,7343,7344,7345
feature_0,-1.043680,0.024369,0.615455,-0.085438,-1.950661,-0.517000,-2.692529,-1.076198,-0.173833,0.051514
feature_1,-1.990359,1.052022,-0.135408,-0.087211,-0.684208,0.062871,-2.081917,-0.133125,-0.621593,1.548708
feature_2,-1.616674,0.669025,5.117934,1.508050,-2.998303,0.752382,-0.332134,0.717059,0.298679,-1.236423
feature_3,0.514712,-1.276948,-3.700463,-1.036884,1.793797,0.061186,0.162031,0.102429,-2.520496,3.029865
feature_4,-0.294449,-0.629200,-3.071781,-0.348713,0.138640,0.112934,-4.167281,-3.733083,0.181639,2.231804
feature_5,-0.168076,-0.723463,-0.870718,1.042102,0.215236,1.553327,-1.498722,-2.430186,0.422883,0.434388
feature_6,-0.737200,1.141347,0.541557,1.173786,-0.266201,0.655819,0.044746,-0.410455,2.391613,-4.436130
feature_7,2.205685,-2.237312,0.961470,-0.504714,-1.607260,-1.007727,-2.162316,-2.942299,0.249525,1.747591
feature_8,-0.822867,2.075696,1.130719,-0.408578,-0.450379,-1.329416,0.797892,1.209964,-0.867040,-2.973186
feature_9,-2.207886,0.502049,-2.539167,0.283530,0.112774,1.045324,-3.580595,-2.117530,-0.597481,1.975224


# Tratando o DataFrame

In [7]:
# Variavel Dependente
var_dep = 'target'
y = df[var_dep]
display(y.head())

x = df.drop(var_dep, axis=1)

0    0
1    1
2    0
3    1
4    0
Name: target, dtype: int32

In [8]:
# DUMMYRIZAÇÃO
colunas_categoricas = []
colunas_binarias = []
colunas_mais3_categorias = []

for coluna in x.columns:
    if df[coluna].dtype == 'O':
        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_mais3_categorias.append(coluna)

############################################################################################
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_categoricas), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

# Remover prefixo 'remainder__' das colunas que não foram transformadas
#novos_nomes_colunas = [nome.replace('remainder__', '') for nome in novos_nomes_colunas]

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 

############################################################################################
# Normalização (scaling entre 0 e 1) com MinMaxScaler ******************************
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
#x = normalizacao.fit_transform(x)
# df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

# Padronização (média 0 e desvio padrão 1) com StandardScaler **********************
from sklearn.preprocessing import StandardScaler
padronizacao = StandardScaler()
#x = padronizacao.fit_transform(x)
# df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()

############################################################################################
# DEFININDO A VARIAVEL DEPENDENTE
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

############################################################################################
#backups
x_inteiro = x
y_inteiro = y

# DIVIDINDO BASE EM TREINO E TESTE
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = 5) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

print(x_treino.shape, x_teste.shape)
print(y_treino.shape, y_teste.shape)

(5509, 10) (1837, 10)
(5509,) (1837,)


In [9]:
print(pd.Series(y_treino).value_counts())
print(pd.Series(y_teste).value_counts())

0    5210
1     299
Name: count, dtype: int64
0    1737
1     100
Name: count, dtype: int64


# Instanciando modelos

In [10]:
sem_random = 3141592

import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier,
                              BaggingClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier, HistGradientBoostingClassifier)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import (LogisticRegression, PassiveAggressiveClassifier,
                                  RidgeClassifier, RidgeClassifierCV, SGDClassifier, Perceptron)
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

# Definir a semente aleatória
sem_random = 3141592

# Criar dicionário com todos os modelos
model_dict = {
    # Modelos de Base
    # 'DummyClassifier': DummyClassifier(),

    # Modelos Baseados em Árvores
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=sem_random),
    'ExtraTreeClassifier': ExtraTreeClassifier(random_state=sem_random),
    'RandomForestClassifier': RandomForestClassifier(random_state=sem_random),
    'ExtraTreesClassifier': ExtraTreesClassifier(random_state=sem_random),
    'BaggingClassifier': BaggingClassifier(random_state=sem_random),
    
    # Modelos de Boosting
    'AdaBoostClassifier': AdaBoostClassifier(random_state=sem_random),
    'GradientBoostingClassifier': GradientBoostingClassifier(random_state=sem_random),
    'HistGradientBoostingClassifier': HistGradientBoostingClassifier(random_state=sem_random),
    'XGBClassifier': XGBClassifier(random_state=sem_random),
    'LGBMClassifier': LGBMClassifier(random_state=sem_random),
    # 'CatBoostClassifier': CatBoostClassifier(verbose=0),

    # Modelos Lineares
    'LogisticRegression': LogisticRegression(random_state=sem_random),
    'PassiveAggressiveClassifier': PassiveAggressiveClassifier(random_state=sem_random),
    'RidgeClassifier': RidgeClassifier(random_state=sem_random),
    'RidgeClassifierCV': RidgeClassifierCV(),
    'SGDClassifier': SGDClassifier(random_state=sem_random),
    'Perceptron': Perceptron(random_state=sem_random),

    # Modelos Baseados em Instâncias
    'KNeighborsClassifier': KNeighborsClassifier(),
    'NearestCentroid': NearestCentroid(),

    # SVM e Modelos Baseados em Hiperplanos
    'SVC': SVC(random_state=sem_random, probability=True), # SVM 
    'LinearSVC': LinearSVC(random_state=sem_random),
    
    # Modelos Probabilísticos e Análise Discriminante
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'MultinomialNB': MultinomialNB(),
    'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis(),
    'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis(),

    # Modelos Semi-Supervisionados
    'LabelSpreading': LabelSpreading(),
    'LabelPropagation': LabelPropagation(),

    # Outros Modelos
    'CalibratedClassifierCV': CalibratedClassifierCV()#,

    # Modelos de Redes Neurais
    # 'MLPClassifier': MLPClassifier(random_state=sem_random),
}

# Treinando modelos

In [11]:
model_dict.items()

dict_items([('DummyClassifier', DummyClassifier()), ('DecisionTreeClassifier', DecisionTreeClassifier(random_state=3141592)), ('ExtraTreeClassifier', ExtraTreeClassifier(random_state=3141592)), ('RandomForestClassifier', RandomForestClassifier(random_state=3141592)), ('ExtraTreesClassifier', ExtraTreesClassifier(random_state=3141592)), ('BaggingClassifier', BaggingClassifier(random_state=3141592)), ('AdaBoostClassifier', AdaBoostClassifier(random_state=3141592)), ('GradientBoostingClassifier', GradientBoostingClassifier(random_state=3141592)), ('HistGradientBoostingClassifier', HistGradientBoostingClassifier(random_state=3141592)), ('XGBClassifier', XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None

In [12]:
import time

print('*'*30, 'NORMAL', '*'*30)
# Treinando todos os modelos
tab_modelo_treinados = pd.DataFrame(columns=['Modelo', 'Status', 'Tempo'])
for model_name, model_instance in model_dict.items():
    # print(model_name)
    try:
        start_time = time.time()
        model_instance.fit(x_treino, y_treino)
        end_time = time.time()
        training_time = end_time - start_time
        tab_modelo_treinados.loc[len(tab_modelo_treinados)] = [model_name, 'OK', training_time]
    except Exception as e:
        tab_modelo_treinados.loc[len(tab_modelo_treinados)] = [model_name, 'Erro', np.nan]
        #del model_dict[model_name] # excluindo do dicionario # não pode modificar o dicionario durante iteracao

display(tab_modelo_treinados)

****************************** NORMAL ******************************
[LightGBM] [Info] Number of positive: 299, number of negative: 5210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 5509, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054275 -> initscore=-2.857892
[LightGBM] [Info] Start training from score -2.857892


,Modelo,Status,Tempo
0,DummyClassifier,OK,0.007653
1,DecisionTreeClassifier,OK,0.395736
2,ExtraTreeClassifier,OK,0.034333
3,RandomForestClassifier,OK,8.719990
4,ExtraTreesClassifier,OK,3.430055
5,BaggingClassifier,OK,2.471773
6,AdaBoostClassifier,OK,2.098010
7,GradientBoostingClassifier,OK,8.866046
8,HistGradientBoostingClassifier,OK,1.672029
9,XGBClassifier,OK,0.517909


In [34]:
tab_modelo_treinados.sort_values(by = 'Tempo', ascending = False)

,Modelo,Status,Tempo
11,CatBoostClassifier,OK,14.844363
30,MLPClassifier,OK,11.298296
7,GradientBoostingClassifier,OK,8.866046
3,RandomForestClassifier,OK,8.719990
20,SVC,OK,6.873408
4,ExtraTreesClassifier,OK,3.430055
5,BaggingClassifier,OK,2.471773
28,LabelPropagation,OK,2.186346
27,LabelSpreading,OK,2.098110
6,AdaBoostClassifier,OK,2.098010


# Analisando as probabilidades geradas pelos modelos

In [14]:
def adicionar_previsoes(x_teste, y_teste, **modelos):
    # Inicializa o DataFrame com a coluna `y`
    df = pd.DataFrame({'y': y_teste})

    # Para cada modelo, gera previsões e adiciona ao DataFrame
    for nome, modelo in modelos.items():
        try:
        #if hasattr(modelo, "predict_proba"):
            df[nome] = modelo.predict_proba(x_teste)[:, 1] * 100
        except:
            print('erro:', nome)

    return df

previsoes = adicionar_previsoes(x_teste, y_teste, **model_dict)
previsoes = previsoes.reset_index(drop=True)

previsoes.sample(10)

erro: PassiveAggressiveClassifier
erro: RidgeClassifier
erro: RidgeClassifierCV
erro: SGDClassifier
erro: Perceptron
erro: NearestCentroid
erro: LinearSVC
erro: MultinomialNB


,y,DummyClassifier,DecisionTreeClassifier,ExtraTreeClassifier,RandomForestClassifier,ExtraTreesClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,HistGradientBoostingClassifier,...,KNeighborsClassifier,SVC,GaussianNB,BernoulliNB,QuadraticDiscriminantAnalysis,LinearDiscriminantAnalysis,LabelSpreading,LabelPropagation,CalibratedClassifierCV,MLPClassifier
1492,0,5.427482,0.0,0.0,4.0,1.0,10.0,47.478894,1.144984,0.174483,...,0.0,3.410832,4.270576,2.110922,7.473461e-05,2.549722,1.828164e-36,9.209689e-155,3.608320,0.455762
59,0,5.427482,0.0,0.0,8.0,0.0,0.0,48.457115,3.583037,1.450948,...,0.0,5.090842,2.091928,7.775389,6.857934e+00,5.279627,1.064828e-04,6.089441e-11,5.409723,1.848943
1623,0,5.427482,0.0,0.0,6.0,7.0,0.0,47.800740,6.656925,0.812941,...,0.0,3.013710,2.518903,5.199981,1.000000e+02,6.019606,7.357171e-09,9.428941e-22,5.819402,1.988963
1313,0,5.427482,0.0,0.0,0.0,2.0,0.0,47.886190,1.302428,0.062510,...,0.0,1.872992,1.901588,5.725429,7.567627e-06,5.067377,1.775692e-24,8.694068e-94,5.234486,0.568990
917,0,5.427482,0.0,0.0,1.0,2.0,0.0,48.504176,2.021900,0.123019,...,0.0,1.434730,26.802746,12.783214,6.920406e-02,12.741541,4.257803e-25,4.392865e-90,9.260894,0.295018
61,0,5.427482,0.0,0.0,2.0,0.0,0.0,48.010026,2.338457,0.095219,...,0.0,2.528075,4.156058,5.668591,6.108177e-16,3.841863,1.118822e-17,7.942149e-73,4.559619,0.270155
416,0,5.427482,0.0,0.0,1.0,9.0,0.0,47.803869,2.361098,1.221775,...,0.0,4.459404,1.622994,3.340220,1.042796e+01,5.171332,1.784121e-03,5.019639e-06,5.367478,2.621409
22,0,5.427482,0.0,0.0,3.0,8.0,0.0,48.913849,5.034982,3.816883,...,0.0,5.174371,2.762901,7.775389,1.674513e+01,6.252281,9.522275e-07,7.541666e-07,5.959985,4.284634
1155,0,5.427482,0.0,0.0,0.0,0.0,0.0,47.852076,2.547323,0.216571,...,0.0,4.085003,2.773234,5.631757,8.467431e+01,5.753071,5.515897e-06,1.558211e-30,5.604438,2.964799
1337,0,5.427482,0.0,0.0,7.0,3.0,0.0,48.538814,4.213024,0.234834,...,0.0,2.433021,4.283920,3.793646,2.644203e+01,4.546028,2.058948e-17,3.240838e-54,4.861934,0.395949


In [15]:
colunas = list(previsoes.columns)
colunas.remove('y')
px.box(previsoes, y=colunas, color="y")

In [16]:
outliers_df = pd.DataFrame(columns=['Coluna', 'Grupo_y', 'Quantidade de Outliers'])

# Iterar pelos grupos definidos pelas colunas 'y' e 'tipo'
for y_value, group in previsoes.groupby('y'):
    # Iterar sobre cada coluna (exceto 'y' e 'tipo') e calcular o número de outliers para cada grupo
    for coluna in group.columns:
        if coluna not in ['y']:
            # Calcular os quartis e o intervalo interquartil (IQR)
            Q1 = group[coluna].quantile(0.25)
            Q3 = group[coluna].quantile(0.75)
            IQR = Q3 - Q1

            # Definir limites inferiores e superiores para detectar outliers
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR

            # Contar quantos valores estão fora dos limites definidos
            outliers_count = group[(group[coluna] < limite_inferior) | (group[coluna] > limite_superior)].shape[0]

            # Adicionar a contagem de outliers ao DataFrame final
            outliers_df.loc[len(outliers_df)] = [coluna, y_value, outliers_count]

# Exibir o DataFrame de outliers
#display(outliers_df)

px.bar(outliers_df, 
       x='Coluna', 
       y='Quantidade de Outliers', 
       facet_col='Grupo_y', 
       text='Quantidade de Outliers')

In [17]:
lista_modelos = list(previsoes.columns)
lista_modelos.remove('y')#; lista_modelos.remove('tipo')
lista_modelos

fd = pd.DataFrame(columns=['modelo', 'tipo', 'threshold', 'tn', 'fp', 'fn', 'tp'])

for modelo in lista_modelos:
    fdzinho = aval_modelo_corte_tabela(y_teste, np.array(previsoes[modelo]))
    fdzinho['modelo'] = modelo

    fd = pd.concat([fd, fdzinho])

# Substituir valores NaN por 0
#fd.fillna(0, inplace=True)
fd = fd.reset_index(drop=True)
######################################################################################

# Inicializar as colunas das métricas com NaN
metricas_de_aval = ['acuracia', 'precisao', 'sensibilidade', 'especificidade', 'f1',
                    'valor_pre_posi', 'valor_pre_neg', 'taxa_falsos_positivos', 'taxa_falsos_negativos',
                    'fdr', 'fo_r', 'indice_youden', 'coef_matthews', 'fb_score']
fd[metricas_de_aval] = fd[metricas_de_aval].round(3)
colunas_uteis = ['modelo', 
                 'threshold', 
                 'tn', 'fp', 'fn', 'tp', 
                 'precisao', 'sensibilidade', 
                 'f1', 'fb_score']
fd = fd[colunas_uteis]
fd = fd.sort_values(by='fb_score', ascending=False)
fd.head(20)

,modelo,threshold,tn,fp,fn,tp,precisao,sensibilidade,f1,fb_score
199,MLPClassifier,20,1707,30,56,44,0.595,0.44,0.506,0.478
198,MLPClassifier,10,1625,112,46,54,0.325,0.54,0.406,0.449
28,RandomForestClassifier,20,1698,39,59,41,0.512,0.41,0.456,0.437
200,MLPClassifier,30,1725,12,64,36,0.750,0.36,0.486,0.429
201,MLPClassifier,40,1731,6,66,34,0.850,0.34,0.486,0.417
126,SVC,10,1696,41,61,39,0.488,0.39,0.433,0.416
37,ExtraTreesClassifier,20,1700,37,62,38,0.507,0.38,0.434,0.412
81,XGBClassifier,10,1697,40,62,38,0.487,0.38,0.427,0.408
90,LGBMClassifier,10,1678,59,60,40,0.404,0.40,0.402,0.401
99,CatBoostClassifier,10,1644,93,56,44,0.321,0.44,0.371,0.395


In [18]:
fd['threshold'] = fd['threshold'].astype('int')

# selecionando apenas os 6 melhores

In [19]:
print(list(fd['modelo'].unique()[:6]))

['MLPClassifier', 'RandomForestClassifier', 'SVC', 'ExtraTreesClassifier', 'XGBClassifier', 'LGBMClassifier']


In [20]:
qtd_de_modelos = 6

###################################################################################################
modelos_escolhidos_normal = list(fd['modelo'].unique()[:qtd_de_modelos])
#modelos_escolhidos_normal.append('y')
previsoes_temp = previsoes[modelos_escolhidos_normal]

lista_modelos_normal = list(previsoes_temp.columns)
#lista_modelos_normal.remove('y')

###################################################################################################
previsoes = pd.concat([previsoes_temp, pd.DataFrame({'y':y_teste})], axis=1)
display(previsoes.sample(5))

lista_modelos = list(previsoes.columns)
lista_modelos.remove('y')

,MLPClassifier,RandomForestClassifier,SVC,ExtraTreesClassifier,XGBClassifier,LGBMClassifier,y
1475,1.267775,1.0,2.910741,1.0,0.209130,0.317981,0
698,4.526414,0.0,3.559537,2.0,0.372636,0.291743,0
1,6.695380,4.0,3.319905,4.0,1.538813,2.087165,0
1730,8.903317,2.0,3.803693,3.0,0.399944,0.957793,0
321,7.268875,0.0,4.549357,12.0,0.377360,0.604872,0


In [21]:
outliers_df = pd.DataFrame(columns=['Coluna', 'Grupo', 'Quantidade de Outliers'])

for y_value, group in previsoes.groupby('y'):
    for coluna in group.columns:
        if coluna != 'y':
            Q1 = group[coluna].quantile(0.25)
            Q3 = group[coluna].quantile(0.75)
            IQR = Q3 - Q1

            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR

            outliers_count = group[(group[coluna] < limite_inferior) | (group[coluna] > limite_superior)].shape[0]

            outliers_df.loc[len(outliers_df)] = [coluna, y_value, outliers_count]

#display(outliers_df)

px.bar(outliers_df, 
       x='Coluna', 
       y='Quantidade de Outliers', 
       facet_col='Grupo', 
       text='Quantidade de Outliers')

# Gerando listas de todas combinaçoes dos 6 modelos pegando de 3 em 3

In [22]:
import math

# Número total de itens (n)
n = qtd_de_modelos
print(n)
# Número de itens escolhidos por combinação (k)
k = 3

# Calcula o número de combinações
print(math.comb(n, k))

6
20


In [23]:
from itertools import combinations

# Gerando todas as combinações de 3 itens
combinacoes_modelos = list(combinations(lista_modelos, 3))

# Voltando a analise

In [24]:
colunas = list(previsoes.columns)
colunas.remove('y')
px.box(previsoes, y=colunas, color="y")

In [25]:
outliers_df = pd.DataFrame(columns=['Coluna', 'Grupo', 'Quantidade de Outliers'])

for y_value, group in previsoes.groupby('y'):
    for coluna in group.columns:
        if coluna != 'y':
            Q1 = group[coluna].quantile(0.25)
            Q3 = group[coluna].quantile(0.75)
            IQR = Q3 - Q1

            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR

            outliers_count = group[(group[coluna] < limite_inferior) | (group[coluna] > limite_superior)].shape[0]

            outliers_df.loc[len(outliers_df)] = [coluna, y_value, outliers_count]

#display(outliers_df)

px.bar(outliers_df, 
       x='Coluna', 
       y='Quantidade de Outliers', 
       facet_col='Grupo', 
       text='Quantidade de Outliers')

# ================================================================

# Analisando combinações de thresholds

In [26]:
from itertools import combinations

# Gerando todas as combinações de 3 itens
combinacoes_modelos = list(combinations(lista_modelos, 3))

In [27]:
len(combinacoes_modelos)

20

In [28]:
grade_thresholds = pd.DataFrame()

for i in range(len(combinacoes_modelos)):
    print('combinacao n:', i)

    lista = list(combinacoes_modelos[i])
    nome_combinacao = "\n".join(lista)

    modelos_escolhidos = lista
    modelos_escolhidos.append('y')

    fd_previsoes = previsoes[modelos_escolhidos]

    grade_thresholds_temp = gridsearch_mult_models_threshold(fd_previsoes, beta=1.5, linspace_thresholds = np.arange(0, 100, 10))
    grade_thresholds_temp['nome_comp'] = nome_combinacao
    grade_thresholds_temp['m1'] = lista[0]
    grade_thresholds_temp['m2'] = lista[1]
    grade_thresholds_temp['m3'] = lista[2]

    grade_thresholds = pd.concat([grade_thresholds, grade_thresholds_temp])

grade_thresholds = grade_thresholds.round(2)
grade_thresholds_bck = grade_thresholds
print('finish')

combinacao n: 0
combinacao n: 1
combinacao n: 2
combinacao n: 3
combinacao n: 4
combinacao n: 5
combinacao n: 6
combinacao n: 7
combinacao n: 8
combinacao n: 9
combinacao n: 10
combinacao n: 11
combinacao n: 12
combinacao n: 13
combinacao n: 14
combinacao n: 15
combinacao n: 16
combinacao n: 17
combinacao n: 18
combinacao n: 19
finish


In [29]:
colunas_uteis = [#'nome_comp', 
                 'threshold', 
                 'tn', 'fp', 'fn', 'tp', 
                 'precisao', 'sensibilidade', 
                 'f1', 'fb_score',
                 0, 1, 2,
                 'm1','m2','m3'
                 ]
grade_thresholds = grade_thresholds[colunas_uteis]
grade_thresholds = grade_thresholds.sort_values('fb_score', ascending=False)
grade_thresholds = grade_thresholds.reset_index()
grade_thresholds.head(30)

,index,threshold,tn,fp,fn,tp,precisao,sensibilidade,f1,fb_score,0,1,2,m1,m2,m3
0,250,"(20, 50, 0)",1707,30,56,44,0.59,0.44,0.51,0.48,20,50,0,MLPClassifier,RandomForestClassifier,LGBMClassifier
1,280,"(20, 80, 0)",1707,30,56,44,0.59,0.44,0.51,0.48,20,80,0,MLPClassifier,RandomForestClassifier,XGBClassifier
2,250,"(20, 50, 0)",1706,31,56,44,0.59,0.44,0.50,0.48,20,50,0,MLPClassifier,SVC,ExtraTreesClassifier
3,270,"(20, 70, 0)",1707,30,56,44,0.59,0.44,0.51,0.48,20,70,0,MLPClassifier,SVC,ExtraTreesClassifier
4,204,"(20, 0, 40)",1706,31,56,44,0.59,0.44,0.50,0.48,20,0,40,MLPClassifier,XGBClassifier,LGBMClassifier
5,290,"(20, 90, 0)",1707,30,56,44,0.59,0.44,0.51,0.48,20,90,0,MLPClassifier,RandomForestClassifier,XGBClassifier
6,205,"(20, 0, 50)",1706,31,56,44,0.59,0.44,0.50,0.48,20,0,50,MLPClassifier,XGBClassifier,LGBMClassifier
7,206,"(20, 0, 60)",1707,30,56,44,0.59,0.44,0.51,0.48,20,0,60,MLPClassifier,XGBClassifier,LGBMClassifier
8,207,"(20, 0, 70)",1707,30,56,44,0.59,0.44,0.51,0.48,20,0,70,MLPClassifier,XGBClassifier,LGBMClassifier
9,208,"(20, 0, 80)",1707,30,56,44,0.59,0.44,0.51,0.48,20,0,80,MLPClassifier,XGBClassifier,LGBMClassifier


# avaliando como o modelo trabalha

In [30]:
#for linha in range(len(grade_thresholds)):
linha = grade_thresholds.index[1]
m1, m2, m3 = grade_thresholds['m1'][linha], grade_thresholds['m2'][linha], grade_thresholds['m3'][linha]
############print(m1, m2, m3)

####################################################################################
corte1, corte2, corte3 = grade_thresholds[0][linha], grade_thresholds[1][linha], grade_thresholds[2][linha]
############print(corte1, corte2, corte3)

####################################################################################
previsoes_filter0 = previsoes[['y', m1, m2, m3]]
previsoes_filter = previsoes_filter0
############previsoes_filter.sample(5)

####################################################################################
previsoes_filter['m1_bin'] = np.where(previsoes_filter[m1] > corte1, 1, 0)
previsoes_filter['m2_bin'] = np.where(previsoes_filter[m2] > corte2, 1, 0)
previsoes_filter['m3_bin'] = np.where(previsoes_filter[m3] > corte3, 1, 0)

previsoes_filter['decisao_final'] = previsoes_filter['m1_bin'] + previsoes_filter['m2_bin'] + previsoes_filter['m3_bin']
previsoes_filter['decisao_final'] = np.where(previsoes_filter['decisao_final'] >= 2, 1, 0)
############previsoes_filter.sample(5)

####################################################################################

import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = previsoes_filter0['y'],
                   colorscale = [[0,'grey'], [1,'red']]),
        dimensions = list([
            dict(range = [0, 100],
                label = m1, values = previsoes_filter0[m1]),
            dict(range = [0, 100],
                label = m2, values = previsoes_filter0[m2]),
            dict(range = [0, 100],
                label = m3, values = previsoes_filter0[m3]),
            dict(range = [0, 1],
                #constraintrange = [0.5,1],
                label = 'y', values = previsoes_filter0['y'])
        ])
    )
)
fig.show()
####################################################################################


In [31]:
print(corte1, corte2, corte3)

20 80 0
